## 层和块

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256),
                    nn.ReLU(),
                    nn.Linear(256, 10))

X = torch.randn(2, 20)
net(X)

tensor([[-0.2820,  0.2162, -0.1494, -0.3162,  0.3441, -0.3098,  0.4643, -0.1026,
         -0.1016, -0.0890],
        [-0.2776,  0.0054, -0.3326, -0.1417,  0.2238, -0.3392, -0.1154,  0.2996,
          0.7091,  0.2328]], grad_fn=<AddmmBackward0>)

## 自定义块

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X)

tensor([[ 1.0084e-02,  3.7609e-02,  1.3886e-01, -2.7226e-01, -4.1112e-01,
          3.2152e-02, -5.4958e-02, -3.4383e-01, -1.5365e-01,  3.2510e-01],
        [ 2.5593e-01, -2.1896e-01,  3.3055e-02, -3.1948e-01, -5.6147e-01,
          3.5116e-01,  2.2873e-01, -3.7952e-01,  2.0768e-01, -2.4989e-04]],
       grad_fn=<AddmmBackward0>)

## nn.Sequential()的实现

In [20]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super(MySequential, self).__init__()
        for block in args:
            # self._modules是一个OrderedDict
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256),
                   nn.ReLU(),
                   nn.Linear(256, 10))
net(X)

tensor([[ 0.0279,  0.3589, -0.1436, -0.3187, -0.3421,  0.2063,  0.0866,  0.2696,
         -0.1653, -0.5540],
        [-0.0469, -0.0095, -0.4558, -0.1489, -0.1605,  0.0008,  0.0477, -0.2529,
          0.3153, -0.0777]], grad_fn=<AddmmBackward0>)

## 在正向传播函数中执行代码

In [22]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super(FixedHiddenMLP, self).__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.1250, grad_fn=<SumBackward0>)

## 混合搭配各种组合块方法

In [25]:
class NestMLP(nn.Module):
    def __init__(self):
        super(NestMLP, self).__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())  # nn.Sequential()中也可以嵌套模型
chimera(X)

tensor(0.2691, grad_fn=<SumBackward0>)

## 参数管理

In [26]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0906],
        [-0.0662]], grad_fn=<AddmmBackward0>)

In [27]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [34]:
print(net[2].state_dict())
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net[2].weight.grad)

OrderedDict([('weight', tensor([[-0.1754, -0.1056, -0.3198, -0.1930, -0.1077, -0.2806, -0.3481,  0.2747]])), ('bias', tensor([-0.1104]))])
<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1104], requires_grad=True)
tensor([-0.1104])
None


## 一次性访问所有参数

In [39]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])  # *表示解包
print(*[(name, param.shape) for name, param in net.named_parameters()])  # 把整个网络的参数全部拿出来

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [43]:
# 注意named_parameters()和state_dict()的区别，一个返回的是generator，一个返回的是OrderedDict
net.state_dict()["2.bias"].data

tensor([-0.1104])

## 从嵌套块中收集参数

In [45]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.0569],
        [0.0569]], grad_fn=<AddmmBackward0>)

In [46]:
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

## 内置初始化

In [48]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0056,  0.0008,  0.0187,  0.0056]), tensor(0.))

In [49]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

## 对某些块应用不同的初始化方法

In [52]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data[0])

tensor([-0.5622, -0.3734,  0.2797,  0.0089])
tensor([42., 42., 42., 42., 42., 42., 42., 42.])


## 自定义初始化

In [60]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])  # 只看weight
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5  # 保留绝对值>=5的权重

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  6.6617, -0.0000,  9.2943],
        [ 0.0000,  8.8055, -7.5792,  0.0000]], grad_fn=<SliceBackward0>)

In [61]:
# 更简单暴力的方法
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  7.6617,  1.0000, 10.2943])

## 某些层之间实现参数共享 -- 参数绑定

In [65]:
shared = nn.Linear(8, 8)  # 先把需要shared的层定义出来
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0] = 800  # 修改第二层的weight
print(net[2].weight.data[0] == net[4].weight.data[0])  # 同样第四层的weight也会发生变化

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 自定义层

In [66]:
import torch
import torch.nn.functional as F
from torch import nn

In [68]:
# 自定义层和自定义网络本质上没有区别
class CenteredLayer(nn.Module):
    def __init__(self):
        super(CenteredLayer, self).__init__()

    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [70]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(-4.1910e-09, grad_fn=<MeanBackward0>)

## 带参数的图层

In [73]:
# 自定义一个线性层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super(MyLinear, self).__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

dense = MyLinear(5, 3)
dense.weight.data

tensor([[ 0.4946,  0.8618, -1.4818],
        [-2.2215,  1.9964,  0.2007],
        [ 1.8015, -0.3722,  0.4262],
        [-0.8105,  0.3027, -1.5462],
        [-0.3965, -0.6675, -0.7849]])

In [74]:
dense(torch.randn(2, 5))

tensor([[3.4650, 0.0000, 0.0000],
        [0.0000, 0.5436, 0.0000]])

In [75]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[ 2.9545],
        [12.7936]])

## 读写文件

In [76]:
import torch
import torch.nn.functional as F
from torch import nn

In [77]:
x = torch.arange(4)
torch.save(x, 'x-file')  # 保存一个张量

x2 = torch.load('x-file')  # 加载一个张量
x2

tensor([0, 1, 2, 3])

In [78]:
# 存储一个张量列表
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [79]:
# 存储成一个字典
my_dict = {'x': x, 'y': y}
torch.save(my_dict, 'my_dict')
my_dict2 = torch.load('my_dict')
my_dict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [81]:
# 加载和保存模型参数
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))

net = MLP()
X = torch.randn(size=(20, 20))
Y = net(X)

In [86]:
torch.save(net.state_dict(), "mlp.params")  # 把模型的参数存起来即可

In [87]:
clone = MLP()
clone.load_state_dict(torch.load("mlp.params"))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [88]:
# 验证参数是否一样
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True,